# DM-J4340-2EC 减速电机

电机相关信息

```
电机大扭矩输出时，会出现发热的情况，请注意避免烫伤。
使用上位机串口连接，电机驱动器会保存设置，而can通讯需要设置保存参数，默认不保存，同时还会将以下五个变量置0
    ：位置命令值、速度命令值、扭矩命令值（MIT）、kp（MIT）、kd（MIT）
上位机串口点击“写参数”按钮芯片会自动复位，因此在使用“写参数”时请确保电机处于“失能”状态
支持can总线进行上位机调试
红灯闪烁表示异常状态
CAN通讯ID号在串口调试那一步会显示
```

疑问
```
双编码器是啥情况
位置速度模式的内部算法
can通讯如何实现
上位机怎么个通信和调试
```

## 控制模式
### 速度模式（控制速度）：
<p align="center">
  <a id="速度模式"></a>
  <img src="./速度模式.png" width="40%">
</p>

<p align="center">
  图1 速度模式
</p>

v_des 单位为 rad/s，数据类型为 float，如需使用调试助手自动计算参数，则需要
设置阻尼因子为非 0 正数，通常情况下取值在 2.0~10.0，过小的阻尼因子会带来速度的震荡
以及较大的过冲，过大的阻尼因子则会带来较长的上升时间，推荐的设定值为 4.0。

### 位置速度模式（控制位置）：
<p align="center">
  <a id="位置速度模式"></a>
  <img src="./位置速度模式.png" width="40%">
</p>

<p align="center">
  图2 位置速度模式
</p>

其中 p_des 是目标位置，v_des 是最大速度限制，用于约束位置环速度的输出，单位分别为 rad 和 rad/s，数据类型为 float；若控制过程中产生额外振荡，可通过提高加/减速度参数进行抑制；阻尼因子必须设置为非
0 的正数

### MIT模式（控制扭矩）
MIT模式可以看做直接控制电磁转矩，根据 MIT 模式可以衍生出多种控制模式，如 kp=0,kd 不为 0 时， 给定 v_des 即可实现匀
速转动;kp=0,kd=0，给定 t_ff 即可实现给定扭矩输出。
注意：对位置进行控制时， kd 不能赋 0，否则会造成电机震荡，甚至失控。
```
可以了解内部实现
```
### 力位混控模式（约束扭矩）
而力位混控，则是在速度环输出后加了“电流（=转矩）饱和/限制/调制”，因为工业伺服的第一目标是 稳定、可靠，显式力矩控制对建模、采样、参数依赖很强，故工业常用该模式

## CAN 通讯
串口（UART）= 点对点、低速、配置/调试用  
CAN = 多节点、实时性强、抗干扰、运动控制用

标准 CAN 帧整体结构（逻辑）：
| SOF | ID(11) | RTR | IDE | r0 | DLC | DATA(0~8B) | CRC | ACK | EOF |
- SOF（Start of Frame）：STM32 CAN 外设寄存器里 没有 SOF 位，无法设置，是硬件层的电平变化
- Identifier（ID，11 位）
- DLC（Data Length Code）
- DATA




## 调试流程
连接电机的串口（需要USB转串口）  
连接电脑，打开上位机，给电机上电，首次上电，上位机会有调试信息，这里面的信息很有用（需电源转化模块以及电源）  
电机侧编码器校准（跳过）
参数标定（跳过）
输出轴编码器校准（跳过）在电机更换驱动板，异常震动等非正常状态才校验  
上位机读、写参数，设置 CAN 相关参数，进行提前保存  
调试，对电机进行控制（需要USB转CAN接口）  
同时可以使用USB转CAN助手，验证 “我发的帧对不对”

需要：电源转接板，XT30 2*2双端线，USB转CAN模块

## 参考内容

[减速电机说明书](./DM-J4340-2EC%20V1.1减速电机说明书(V4版本驱动).pdf)

[调试工具用户手册](./DM调试工具用户手册_2025.3.28_V1.0.pdf)